In [1]:
!pip install pyspark

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import sys
import re
from pyspark import SparkContext, SparkConf
import numpy as np
import time as time
from operator import itemgetter

In [2]:
# create Spark context with necessary configuration
sc = SparkContext("local","PySpark HITS algorithm")

In [3]:
# import the data
filepath = '/content/drive/My Drive/Coursework/CS 5683_Big_Data/Assignment-6/email-Eu-core.txt'
links = sc.textFile(filepath)

links_formatted = links.map(lambda x: (int(x.split(' ')[0]),int(x.split(' ')[1])))

#transpose
links_formatted_t = links.map(lambda x: (int(x.split(' ')[1]),int(x.split(' ')[0])))

In [4]:
links_formatted_list = links_formatted.map(lambda x: (x[0],[x[1]]))
link_formatted_dict = links_formatted_list.reduceByKey(lambda a,b: a+b)

#transpose
links_formatted_list_t = links_formatted_t.map(lambda x: (x[0],[x[1]]))
link_formatted_dict_t = links_formatted_list_t.reduceByKey(lambda a,b: a+b)

print('The number of nodes in L is ',link_formatted_dict.count())
print('The number of nodes in L.T is ',link_formatted_dict_t.count())


The number of nodes in L is  868
The number of nodes in L.T is  991


In [5]:
hubs = links_formatted.flatMap(lambda x: x).distinct().map(lambda x: (x, 1.0))
n_nodes = hubs.count()

print('Total number of nodes is ', n_nodes)
elements = list(np.zeros(n_nodes))
rdd = sc.parallelize(elements)

rdd = rdd.zipWithIndex()

def getList(x):
  return_list = list(np.zeros(n_nodes))
  return return_list

matRdd = rdd.map(lambda x: getList(x[0]))

#transpose
matRdd_t = rdd.map(lambda x: getList(x[0]))


Total number of nodes is  1005


In [6]:
def getList_empty(x):
  return_list = []
  return return_list

empty_rdd = rdd.map(lambda x: getList_empty(x[0]))
empty_rdd = empty_rdd.zipWithIndex()
empty_rdd = empty_rdd.map(lambda x: (x[1], x[0]))

def add_two_lists(x):
  if (x[1][0]) == None :
    return (x[0],['A'])
  else:
    return (x[0],x[1][0])

zip_empty = link_formatted_dict.rightOuterJoin(empty_rdd).map(lambda x: add_two_lists(x))
zip_empty_p = zip_empty.repartition(1)

#transpose
zip_empty_t = link_formatted_dict_t.rightOuterJoin(empty_rdd).map(lambda x: add_two_lists(x))
zip_empty_p_t = zip_empty_t.repartition(1)


In [13]:
for a in zip_empty_t.collect(): 
  if a[0] == 78:
    print(a)

(78, [77, 497, 539, 310, 114, 62, 340, 419, 531, 482, 160, 79, 548, 366, 424, 93, 82, 13, 420, 121, 581, 107, 405])


In [7]:
matRdd1 = matRdd.zipWithIndex()
matRdd_zip = matRdd1.map(lambda x: (x[1], x[0]))

#transpose
matRdd_t1 = matRdd_t.zipWithIndex()
matRdd_t_zip = matRdd_t1.map(lambda x: (x[1], x[0]))

l_matrix = zip_empty_p.join(matRdd_zip)
print(l_matrix.count())

#transpose
l_t_matrix = zip_empty_p_t.join(matRdd_t_zip)
print(l_t_matrix.count())

1005
1005


In [11]:
print(l_t_matrix.take(1))

[(4, ([2, 62, 4, 286, 412, 63, 107, 174, 3, 195, 6, 194, 58, 114, 281, 665, 193, 160, 635, 211, 59, 285, 408, 520, 82, 192, 282, 481, 283, 301, 305, 424, 137, 121, 142, 809, 158, 57, 273, 826, 832, 685, 303, 845, 431, 13, 612, 411, 256, 564, 208, 145, 587, 533, 859, 425, 252, 738, 89, 803, 96, 183, 138, 896, 971, 329, 271, 990, 812, 323, 56, 405, 166, 1001], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [8]:
for a in l_t_matrix.collect():
  # print(a[0])
  if a[1][0] == 'A':
    print(a[0])
    print(a[1][0])
    print(a[1][1])

In [ ]:
def form_matrix(x):
  if x[1][0][0] == 'A':
    l1 = [0 for i in range(n_nodes)]
    return (x[0],l1)
  else:
    for k in x[1][0]:
      x[1][1][1][k] = 1
    return (x[0],x[1][1][1])

l_matrix = l_matrix.map(lambda x: form_matrix(x))
print(l_matrix.count())

#transpose
l_t_matrix   = l_t_matrix.map(lambda x: form_matrix(x))
print(l_t_matrix.count())


In [ ]:
hubs_ones = [1 for i in range(n_nodes)]
hubs = sc.parallelize(hubs_ones)
hubs_updated = hubs_ones

def product_sum(x):
  if x[1][1] == None:
    return 0
  else:
    # return (x[0],np.sum(np.array(x[1]*np.array(x[2]))))
    return np.sum(np.array(x[1]*np.array(x[2])))

start0 = time.time()
for i in range(40):
  start = time.time()

  matrix_hubs = l_matrix.map(lambda x: (x[0],x[1],hubs_updated))
  matrix_product_hubs = matrix_hubs.map(lambda x: product_sum(x))

  auths_list = matrix_product_hubs.collect()
  auths_list = [auths_list[i]/max(auths_list) for i in range(len(auths_list))]

  matrix_t_auths = l_t_matrix.map(lambda x: (x[0],x[1],auths_list))
  matrix_t_product_auths = matrix_t_auths.map(lambda x: product_sum(x))

  hubs_updated = matrix_t_product_auths.collect()
  hubs_updated = [hubs_updated[i]/max(hubs_updated) for i in range(len(hubs_updated))]

  print(i, np.round(time.time() - start,4))

print('Total time : ', np.round(time.time() - start0,4))

In [ ]:
tuple_hubs = []
tuple_auths = []

for i in range(0, n_nodes):
  t = (i, hubs_updated[i])
  tuple_hubs.append(t)

  t = (i, auths_list[i])
  tuple_auths.append(t)

sorted_hubs_asc = sorted(tuple_hubs,key=itemgetter(1), reverse=False)
sorted_hubs_dsc = sorted(tuple_hubs,key=itemgetter(1), reverse=True)

sorted_auths_asc = sorted(tuple_auths,key=itemgetter(1), reverse=False)
sorted_auths_dsc = sorted(tuple_auths,key=itemgetter(1), reverse=True)

In [ ]:
print('Top nodes IDs with hub scores')
for i in range(5):
  print("{:<5} {:<5} {:<5} ".format(i+1, sorted_hubs_dsc[i][0], np.round(sorted_hubs_dsc[i][1],4)))

Top nodes IDs with hub scores
1     182   1.0   
2     28    0.8942 
3     543   0.8555 
4     31    0.8101 
5     547   0.7635 


In [ ]:
print('Bottom nodes IDs with hub scores')
for i in range(5):
  print("{:<5} {:<5} {:<5} ".format(i+1, sorted_hubs_asc[i][0], np.round(sorted_hubs_asc[i][1],4)))

Bottom nodes IDs with hub scores
1     289   0.0   
2     407   0.0   
3     464   0.0   
4     488   0.0   
5     491   0.0   


In [ ]:
print('Top nodes IDs with authority scores')
for i in range(5):
  print("{:<5} {:<5} {:<5} ".format(i+1, sorted_auths_dsc[i][0], np.round(sorted_auths_dsc[i][1],4)))

Top nodes IDs with authority scores
1     40    1.0   
2     17    0.7598 
3     656   0.7553 
4     99    0.7333 
5     629   0.6497 


In [ ]:
print('Bottom nodes IDs with authority scores')
for i in range(5):
  print("{:<5} {:<5} {:<5} ".format(i+1, sorted_auths_asc[i][0], np.round(sorted_auths_asc[i][1],4)))

Bottom nodes IDs with authority scores
1     429   0.0   
2     430   0.0   
3     431   0.0   
4     432   0.0   
5     433   0.0   
